# Dynamic Policy Impacts Analysis

This notebook calculates both static and dynamic budgetary impacts for all reform options.
Dynamic scoring incorporates CBO labor supply response elasticities to estimate behavioral effects.

**Note**: This notebook generates the data files used in the revenue impacts analysis. Run this first to update the static and dynamic impact estimates.

In [5]:
# Import necessary libraries
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from reforms import REFORMS
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## Setup: CBO Labor Response Parameters

Define the CBO labor supply elasticities used for dynamic scoring:

In [6]:
# CBO labor supply elasticities
# Note: Using simplified parameters to avoid recursion issues
# Full CBO elasticities can cause circular dependencies in benefit calculations
CBO_LABOR_PARAMS = {
    "gov.simulation.labor_supply_responses.elasticities.income": {
        "2024-01-01.2100-12-31": -0.05
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
        "2024-01-01.2100-12-31": 0.31
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
        "2024-01-01.2100-12-31": 0.28
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
        "2024-01-01.2100-12-31": 0.27
    },
    # Substitution elasticities commented out due to recursion issues
    # These create circular dependencies with SNAP and other benefit calculations
    # "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
    #     "2024-01-01.2100-12-31": 0.31
    # },
    # ... (other substitution elasticities)
}

print("CBO labor supply elasticities configured")
print(f"Income elasticity: {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.income']['2024-01-01.2100-12-31']}")
print(f"Primary earner substitution (decile 1): {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1']['2024-01-01.2100-12-31']}")
print(f"Secondary earner substitution: {CBO_LABOR_PARAMS['gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary']['2024-01-01.2100-12-31']}")

## Helper Functions

In [ ]:
def create_dynamic_reform(base_reform_dict, labor_params=CBO_LABOR_PARAMS):
    """
    Combine a base reform with CBO labor supply elasticities.
    
    Args:
        base_reform_dict: Dictionary of reform parameters  
        labor_params: Dictionary of labor supply elasticity parameters
    
    Returns:
        Reform object with combined parameters
    """
    # Combine dictionaries (both use DOT notation for date ranges)
    combined_dict = {**base_reform_dict, **labor_params}
    
    return Reform.from_dict(combined_dict, country_id="us")


def get_reform_dict(reform_func):
    """
    Extract the underlying parameter dictionary from a reform function.
    
    This works by importing the helper functions from reforms.py that return
    the raw dictionaries before they're wrapped in Reform objects.
    """
    # Import the helper functions that return dicts
    from reforms import (
        eliminate_ss_taxation, tax_85_percent_ss, tax_100_percent_ss,
        extend_senior_deduction, add_ss_tax_credit, eliminate_senior_deduction,
        enable_employer_payroll_tax
    )
    
    # Map each reform function to its underlying dict-returning function(s)
    # This is based on the structure in reforms.py
    reform_func_name = reform_func.__name__
    
    if reform_func_name == "get_option1_reform":
        return eliminate_ss_taxation()
    elif reform_func_name == "get_option2_reform":
        return tax_85_percent_ss()
    elif reform_func_name == "get_option3_reform":
        return {**tax_85_percent_ss(), **extend_senior_deduction()}
    elif reform_func_name == "get_option4_reform":
        return {**tax_85_percent_ss(), **add_ss_tax_credit(500), **eliminate_senior_deduction()}
    elif reform_func_name == "get_option5_reform":
        return {**eliminate_ss_taxation(), **enable_employer_payroll_tax(1.0)}
    elif reform_func_name == "get_option6_reform":
        # Option 6 is more complex with phase-in schedules
        return {
            "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
                "2026-01-01.2100-12-31": True
            },
            "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
                "2026-01-01.2026-12-31": 0.1307,
                "2027-01-01.2027-12-31": 0.2614,
                "2028-01-01.2028-12-31": 0.3922,
                "2029-01-01.2029-12-31": 0.5229,
                "2030-01-01.2030-12-31": 0.6536,
                "2031-01-01.2031-12-31": 0.7843,
                "2032-01-01.2032-12-31": 0.9150,
                "2033-01-01.2100-12-31": 1.0
            },
            "gov.irs.social_security.taxability.rate.base": {
                "2029-01-01.2029-12-31": 0.45,
                "2030-01-01.2030-12-31": 0.40,
                "2031-01-01.2031-12-31": 0.35,
                "2032-01-01.2032-12-31": 0.30,
                "2033-01-01.2033-12-31": 0.25,
                "2034-01-01.2034-12-31": 0.20,
                "2035-01-01.2035-12-31": 0.15,
                "2036-01-01.2036-12-31": 0.10,
                "2037-01-01.2037-12-31": 0.05,
                "2038-01-01.2100-12-31": 0
            },
            "gov.irs.social_security.taxability.rate.additional": {
                "2029-01-01.2029-12-31": 0.80,
                "2030-01-01.2030-12-31": 0.75,
                "2031-01-01.2031-12-31": 0.70,
                "2032-01-01.2032-12-31": 0.65,
                "2033-01-01.2033-12-31": 0.60,
                "2034-01-01.2034-12-31": 0.55,
                "2035-01-01.2035-12-31": 0.50,
                "2036-01-01.2036-12-31": 0.45,
                "2037-01-01.2037-12-31": 0.40,
                "2038-01-01.2038-12-31": 0.35,
                "2039-01-01.2039-12-31": 0.30,
                "2040-01-01.2040-12-31": 0.25,
                "2041-01-01.2041-12-31": 0.20,
                "2042-01-01.2042-12-31": 0.15,
                "2043-01-01.2043-12-31": 0.10,
                "2044-01-01.2044-12-31": 0.05,
                "2045-01-01.2100-12-31": 0
            }
        }
    elif reform_func_name == "get_option7_reform":
        return eliminate_senior_deduction()
    elif reform_func_name == "get_option8_reform":
        return tax_100_percent_ss()
    else:
        raise ValueError(f"Unknown reform function: {reform_func_name}")


def calculate_revenue_impact(reform, year, baseline_income_tax):
    """
    Calculate revenue impact for a given reform and year.
    
    Args:
        reform: Reform object (can be static or dynamic)
        year: Year to calculate impact for
        baseline_income_tax: Pre-computed baseline income tax array
    
    Returns:
        Revenue impact in dollars (positive = revenue gain, negative = revenue loss)
    """
    # Create reformed simulation
    reform_sim = Microsimulation(reform=reform)
    
    # Calculate reformed income tax
    reform_income_tax = reform_sim.calculate("income_tax", map_to="household", period=year)
    
    # JCT convention: reformed - baseline (positive = more revenue)
    revenue_impact = reform_income_tax.sum() - baseline_income_tax.sum()
    
    return revenue_impact


def compute_baselines(years):
    """Pre-compute baselines for all years to avoid redundant calculations.
    
    Args:
        years: List of years to compute baselines for
    
    Returns:
        Dictionary mapping years to baseline income tax arrays
    """
    print("Pre-computing baselines for all years...")
    baselines = {}
    
    for year in years:
        print(f"  Computing baseline for {year}...")
        baseline = Microsimulation()
        baseline_income_tax = baseline.calculate("income_tax", map_to="household", period=year)
        baselines[year] = baseline_income_tax
    
    print("Baseline computation complete!\n")
    return baselines


print("Helper functions defined")

Helper functions defined


## Static Impact Calculation

Calculate static budgetary impacts (without behavioral responses) for all reforms across 2026-2035.
This recalculates the baseline estimates with the latest PolicyEngine US updates.

In [8]:
# Years to analyze
YEARS = list(range(2026, 2036))

# Pre-compute baselines for all years
baselines = compute_baselines(YEARS)

# Storage for results
static_results = []

# Create data directory if it doesn't exist
import os
os.makedirs('../data', exist_ok=True)

print("\n" + "="*80)
print("STATIC IMPACT CALCULATIONS")
print("="*80)
print(f"Analyzing {len(REFORMS)} reforms across {len(YEARS)} years = {len(REFORMS) * len(YEARS)} calculations\n")

for reform_id, reform_config in tqdm(REFORMS.items(), desc="Reforms"):
    reform_name = reform_config['name']
    reform_func = reform_config['func']
    
    print(f"\nProcessing {reform_id}: {reform_name}")
    
    try:
        # Get the static reform (already works with the existing Reform objects from reforms.py)
        static_reform = reform_func()
        
        for year in tqdm(YEARS, desc=f"  Years ({reform_id})", leave=False):
            print(f"  Calculating {year}...", end=' ')
            
            impact = calculate_revenue_impact(static_reform, year, baselines[year])
            
            static_results.append({
                'reform_id': reform_id,
                'reform_name': reform_name,
                'year': year,
                'revenue_impact': impact,
                'scoring_type': 'static'
            })
            
            print(f"${impact/1e9:.2f}B")
        
        # Save checkpoint after each reform completes
        checkpoint_df = pd.DataFrame(static_results)
        checkpoint_df.to_csv('../data/policy_impacts_static_checkpoint.csv', index=False)
        print(f"  ✓ Checkpoint saved ({len(static_results)} records so far)")
        
    except Exception as e:
        print(f"  ✗ ERROR processing {reform_id}: {type(e).__name__}: {e}")
        print(f"  Continuing with next reform...")
        import traceback
        traceback.print_exc()

# Convert to DataFrame
static_df = pd.DataFrame(static_results)

print("\n" + "="*80)
print("STATIC IMPACTS SUMMARY")
print("="*80)

if len(static_df) > 0:
    # Show 10-year totals
    static_totals = static_df.groupby(['reform_id', 'reform_name'])['revenue_impact'].sum() / 1e9
    print("\n10-Year Static Impacts (2026-2035, Billions):")
    for reform_id, total in static_totals.items():
        print(f"  {reform_id[0]}: ${total:,.1f}B")
    
    print(f"\nTotal records: {len(static_df)}")
else:
    print("⚠ No static results calculated")

Pre-computing baselines for all years...
  Computing baseline for 2026...
  Computing baseline for 2027...
  Computing baseline for 2028...
  Computing baseline for 2029...
  Computing baseline for 2030...
  Computing baseline for 2031...
  Computing baseline for 2032...
  Computing baseline for 2033...
  Computing baseline for 2034...
  Computing baseline for 2035...
Baseline computation complete!


STATIC IMPACT CALCULATIONS
Analyzing 8 reforms across 10 years = 80 calculations



Reforms:   0%|          | 0/8 [00:00<?, ?it/s]


Processing option1: Full Repeal of Social Security Benefits Taxation


  Calculating 2026... 

Reforms:   0%|          | 0/8 [00:06<?, ?it/s]


AttributeError: 'ParameterNode' object has no attribute 'update'

## Dynamic Impact Calculation

Calculate dynamic budgetary impacts incorporating CBO labor supply elasticities.
This captures behavioral responses to tax changes.

In [5]:
# Storage for dynamic results
dynamic_results = []

# Create data directory if it doesn't exist
import os
os.makedirs('../data', exist_ok=True)

print("\n" + "="*80)
print("DYNAMIC IMPACT CALCULATIONS")
print("="*80)
print(f"Analyzing {len(REFORMS)} reforms across {len(YEARS)} years = {len(REFORMS) * len(YEARS)} calculations\n")

for reform_id, reform_config in tqdm(REFORMS.items(), desc="Reforms"):
    reform_name = reform_config['name']
    reform_func = reform_config['func']
    
    print(f"\nProcessing {reform_id}: {reform_name}")
    
    # Skip Option 6 for dynamic scoring (too complex)
    if reform_id == "option6":
        print("  ⚠ Skipping Option 6 - complex phase-in not compatible with dynamic scoring")
        continue
    
    try:
        # Get the reform dict and combine with labor elasticities
        reform_dict = get_reform_dict(reform_func)
        dynamic_reform = create_dynamic_reform(reform_dict, CBO_LABOR_PARAMS)
        
        for year in tqdm(YEARS, desc=f"  Years ({reform_id})", leave=False):
            print(f"  Calculating {year}...", end=' ')
            
            impact = calculate_revenue_impact(dynamic_reform, year, baselines[year])
            
            dynamic_results.append({
                'reform_id': reform_id,
                'reform_name': reform_name,
                'year': year,
                'revenue_impact': impact,
                'scoring_type': 'dynamic'
            })
            
            print(f"${impact/1e9:.2f}B")
        
        # Save checkpoint after each reform completes
        checkpoint_df = pd.DataFrame(dynamic_results)
        checkpoint_df.to_csv('../data/policy_impacts_dynamic_checkpoint.csv', index=False)
        print(f"  ✓ Checkpoint saved ({len(dynamic_results)} records so far)")
        
    except Exception as e:
        print(f"  ✗ ERROR processing {reform_id}: {type(e).__name__}: {e}")
        print(f"  Continuing with next reform...")
        import traceback
        traceback.print_exc()

# Convert to DataFrame
dynamic_df = pd.DataFrame(dynamic_results)

print("\n" + "="*80)
print("DYNAMIC IMPACTS SUMMARY")
print("="*80)

if len(dynamic_df) > 0:
    # Show 10-year totals
    dynamic_totals = dynamic_df.groupby(['reform_id', 'reform_name'])['revenue_impact'].sum() / 1e9
    print("\n10-Year Dynamic Impacts (2026-2035, Billions):")
    for reform_id, total in dynamic_totals.items():
        print(f"  {reform_id[0]}: ${total:,.1f}B")
    
    print(f"\nTotal records: {len(dynamic_df)}")
else:
    print("⚠ No dynamic results calculated")



DYNAMIC IMPACT CALCULATIONS
Analyzing 8 reforms across 10 years = 80 calculations



Reforms:   0%|          | 0/8 [00:00<?, ?it/s]


Processing option1: Full Repeal of Social Security Benefits Taxation


  Calculating 2026... 

  ✗ ERROR processing option1: Exception: RecursionError while calculating snap for period 2026-01. The full computation stack is:
  - household_net_income 2026, baseline_lsr_measurement
  - household_benefits 2026, baseline_lsr_measurement
  - snap 2026, baseline_lsr_measurement
  - snap 2026-01, baseline_lsr_measurement
  Continuing with next reform...


Traceback (most recent call last):
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\simulations\simulation.py", line 715, in _calculate
    array = self._run_formula(variable, population, period)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\simulations\simulation.py", line 1005, in _run_formula
    array = formula(population, period, parameters_at)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-us\policyengine_us\variables\gov\usda\snap\snap.py", line 18, in formula
    takes_up = spm_unit("takes_up_snap_if_eligible", period)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\populations\group_population.py", line 38, in __call__
    return super().__call__(variable_name, period, options)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


Processing option2: Taxation of 85% of Social Security Benefits


  Calculating 2026... 

Reforms:  12%|█▎        | 1/8 [23:24<2:43:48, 1404.08s/it]


KeyboardInterrupt: 

## Comparison: Dynamic vs Static

Calculate the difference between dynamic and static scores to show the behavioral feedback effects.

In [ ]:
# Merge static and dynamic results
print("\n" + "="*80)
print("BEHAVIORAL FEEDBACK EFFECTS")
print("="*80)

if len(static_df) == 0:
    print("⚠ No static results to compare")
elif len(dynamic_df) == 0:
    print("⚠ No dynamic results to compare")
else:
    comparison_df = pd.merge(
        static_df[['reform_id', 'reform_name', 'year', 'revenue_impact']].rename(columns={'revenue_impact': 'static_impact'}),
        dynamic_df[['reform_id', 'reform_name', 'year', 'revenue_impact']].rename(columns={'revenue_impact': 'dynamic_impact'}),
        on=['reform_id', 'reform_name', 'year'],
        how='outer'  # Use outer join to keep all results even if one side is missing
    )

    # Calculate difference (dynamic - static)
    # Positive difference = behavioral responses increase revenue relative to static
    # Negative difference = behavioral responses decrease revenue relative to static
    comparison_df['dynamic_feedback'] = comparison_df['dynamic_impact'] - comparison_df['static_impact']

    # Calculate percentage difference
    comparison_df['feedback_pct'] = (comparison_df['dynamic_feedback'] / comparison_df['static_impact'].abs()) * 100

    # Show 10-year feedback effects (only for reforms with both static and dynamic)
    complete_reforms = comparison_df.dropna(subset=['static_impact', 'dynamic_impact'])
    
    if len(complete_reforms) > 0:
        feedback_summary = complete_reforms.groupby(['reform_id', 'reform_name']).agg({
            'static_impact': 'sum',
            'dynamic_impact': 'sum',
            'dynamic_feedback': 'sum'
        }) / 1e9

        feedback_summary['feedback_pct'] = (feedback_summary['dynamic_feedback'] / feedback_summary['static_impact'].abs()) * 100

        print("\n10-Year Comparison (2026-2035, Billions):")
        print("-" * 80)
        for (reform_id, reform_name), row in feedback_summary.iterrows():
            print(f"\n{reform_id}: {reform_name}")
            print(f"  Static impact:     ${row['static_impact']:>8,.1f}B")
            print(f"  Dynamic impact:    ${row['dynamic_impact']:>8,.1f}B")
            print(f"  Feedback effect:   ${row['dynamic_feedback']:>8,.1f}B ({row['feedback_pct']:+.1f}%)")

        print(f"\n✓ Comparison complete for {len(feedback_summary)} reforms")
    else:
        print("\n⚠ No reforms have both static and dynamic results to compare")
        comparison_df = pd.DataFrame()  # Empty dataframe

    comparison_df.head()

## Export Results

Save all results to CSV files for use in other analyses and visualizations.

In [ ]:
import os

# Create data directory if it doesn't exist
os.makedirs('../data', exist_ok=True)

print("\n" + "="*80)
print("EXPORTING RESULTS")
print("="*80)

# Export static impacts (this replaces the old policy_impacts.csv)
if len(static_df) > 0:
    static_df.to_csv('../data/policy_impacts_static.csv', index=False)
    print("✓ Exported static impacts to: data/policy_impacts_static.csv")
    print(f"  ({len(static_df)} records)")
    
    # Also export a legacy version for backward compatibility
    static_df.to_csv('../data/policy_impacts.csv', index=False)
    print("✓ Exported static impacts (legacy) to: data/policy_impacts.csv")
else:
    print("⚠ No static impacts to export")

# Export dynamic impacts
if len(dynamic_df) > 0:
    dynamic_df.to_csv('../data/policy_impacts_dynamic.csv', index=False)
    print("✓ Exported dynamic impacts to: data/policy_impacts_dynamic.csv")
    print(f"  ({len(dynamic_df)} records)")
else:
    print("⚠ No dynamic impacts to export")

# Export comparison data
if 'comparison_df' in locals() and len(comparison_df) > 0:
    comparison_df.to_csv('../data/policy_impacts_comparison.csv', index=False)
    print("✓ Exported comparison data to: data/policy_impacts_comparison.csv")
    print(f"  ({len(comparison_df)} records)")
    
    # Export summary statistics
    if 'feedback_summary' in locals() and len(feedback_summary) > 0:
        summary_stats = pd.DataFrame({
            'reform_id': feedback_summary.index.get_level_values(0),
            'reform_name': feedback_summary.index.get_level_values(1),
            'static_10yr_billions': feedback_summary['static_impact'].values,
            'dynamic_10yr_billions': feedback_summary['dynamic_impact'].values,
            'feedback_billions': feedback_summary['dynamic_feedback'].values,
            'feedback_percent': feedback_summary['feedback_pct'].values
        })
        summary_stats.to_csv('../data/policy_impacts_summary.csv', index=False)
        print("✓ Exported summary statistics to: data/policy_impacts_summary.csv")
        print(f"  ({len(summary_stats)} reforms)")
else:
    print("⚠ No comparison data to export")

# Clean up checkpoint files if everything completed successfully
if len(static_df) > 0 and len(dynamic_df) > 0:
    if os.path.exists('../data/policy_impacts_static_checkpoint.csv'):
        os.remove('../data/policy_impacts_static_checkpoint.csv')
        print("\n✓ Removed static checkpoint file (all complete)")
    if os.path.exists('../data/policy_impacts_dynamic_checkpoint.csv'):
        os.remove('../data/policy_impacts_dynamic_checkpoint.csv')
        print("✓ Removed dynamic checkpoint file (all complete)")

print("\n" + "="*80)
print("EXPORT SUMMARY")
print("="*80)
print(f"Static reforms processed: {static_df['reform_id'].nunique() if len(static_df) > 0 else 0}")
print(f"Dynamic reforms processed: {dynamic_df['reform_id'].nunique() if len(dynamic_df) > 0 else 0}")
print(f"Years analyzed: {YEARS[0]}-{YEARS[-1]}")
print("\nAll exports complete!")